# 동기화와 교착 상태

## 동기화 개념
- 공유 자원: 프로세스 혹은 스레드가 공유하는 자원
- 임계 구역: 동시에 실행했을 때, 문제가 발생하는 자원에 접근하는 코드 영역
- 레이스 컨디션: 프로세스 혹은 스레드가 동시에 임계 구역의 코드를 실행하여 문제가 발생하는 상황
- ![](https://myapollo.com.tw/images/interview-question-race-condition/race-condition-sequence-diagram.png)
- 동기화: 레이스 컨디션을 방지하면서, 임계 구역을 관리하기 위한 방법
    - 실행 순서 제어: 프로세스 및 스레드를 올바른 순서로 실행하기
    - 상호 배제: 동시에 접근해서는 안 되는 자원에 하나의 프로세스 및 스레드만 접근하기

## 동기화 기법

### 뮤텍스 락
- 정의: 동시에 접근해서는 안되는 자원에 동시에 접근하지 않도록 만드는 도구
- 원리
    - 임계 구역에 접근하고자 한다면 반드시 Lock을 획득해야 하고,
    - 임계 구역에서의 작업이 끝났다면 락을 해제 해야 한다.
    - ![](https://blog.kakaocdn.net/dna/TyGK5/btqUYfBRe7j/AAAAAAAAAAAAAAAAAAAAAEqS2nOS0WYDbuWvMC9Pnn3HYipRrcVySaDvd6YD3k_U/img.png?credential=yqXZFxpELC7KVnFOS48ylbz2pIh7yKj8&expires=1759244399&allow_ip=&allow_referer=&signature=Tk3k0kqxLyyjMk2cmyV%2FLINq%2B%2FY%3D)

In [4]:
import threading

shared_data = 0           # 공유 데이터
mutex = threading.Lock()  # 뮤텍스(락) 선언

def increment():
    global shared_data
    for _ in range(100000):
        with mutex:       # 락 획득/해제 (RAII 스타일)
            shared_data += 1

def decrement():
    global shared_data
    for _ in range(100000):
        with mutex:
            shared_data -= 1

def main():
    t1 = threading.Thread(target=increment)
    t2 = threading.Thread(target=decrement)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print(f"Final value of shared_data: {shared_data}")

if __name__ == "__main__":
    main()


Final value of shared_data: 0


### 세마포어
- 정의: 공유 자원이 여러 개 있는 상황에서도 적용이 가능한 동기화 도구
- 구성 요소
    - 변수 S: 사용 가능한 공유 자원의 갯수를 나타내는 변수
    - wait() 함수: 임계 구역 진입 전 호출하는 함수
    - signal() 함수: 임계 구역에서 작업이 끝난 후 호출하는 함수
- 원리
    - 임계 구역에 진입할 때
        - S를 1 감소 시킴
        - S의 값이 0보다 작은지 확인 (공유자원이 남아 있는지 확인)
            - 0보다 작다면, 대기 큐에 들어가 대기
            - 0보다 크거나 같다면, 임계 구역에 진입
    - 임계 구역에서 나올 때
        - S를 1 증가 시킴
        - S의 값이 0 이하인지 확인 (공유자원이 남아 있는지 확인)
            - 0보다 작다면, 대기 큐에서 하나를 깨워서 임계 구역에 진입시킴
            - 0보다 크거나 같다면, 아무 작업도 하지 않음
    - ![](https://velog.velcdn.com/images/abcdana/post/2923c854-d07c-4ecd-a809-c6b2609505fa/image.png)
    - ![](https://mblogthumb-phinf.pstatic.net/MjAyMDA3MTFfMzMg/MDAxNTk0NDE5NDEwMDk5.1Zso_Irewd9fkYAIkETC3uMPRKkXwGDWvbeztQn9RMsg.9u_RSLRzrG9a5HT-_nqlbnKXub2GCSfaiB-UOUTkdVcg.PNG.adamdoha/image.png?type=w800)

In [8]:
import threading

# 자원별 공유 데이터
shared_a = 0
shared_b = 0

# 자원별 세마포어(각 자원 동시 2개까지 허용)
sem_a = threading.Semaphore(2)
sem_b = threading.Semaphore(2)

def increment_a():
    global shared_a
    for _ in range(100000):
        sem_a.acquire()          # 자원 A 획득(최대 2개 동시 진입 허용)
        try:
            shared_a += 1        # 자원 A에 대한 작업
        finally:
            sem_a.release()      # 자원 A 해제

def decrement_a():
    global shared_a
    for _ in range(100000):
        sem_a.acquire()
        try:
            shared_a -= 1
        finally:
            sem_a.release()

def increment_b():
    global shared_b
    for _ in range(100000):
        sem_b.acquire()          # 자원 B 획득(최대 2개 동시 진입 허용)
        try:
            shared_b += 1        # 자원 B에 대한 작업
        finally:
            sem_b.release()

def decrement_b():
    global shared_b
    for _ in range(100000):
        sem_b.acquire()
        try:
            shared_b -= 1
        finally:
            sem_b.release()

def main():
    t1 = threading.Thread(target=increment_a)
    t2 = threading.Thread(target=decrement_a)
    t3 = threading.Thread(target=increment_b)
    t4 = threading.Thread(target=decrement_b)

    t1.start(); t2.start(); t3.start(); t4.start()
    t1.join();  t2.join();  t3.join();  t4.join()

    print(f"Final value of shared_a: {shared_a}")
    print(f"Final value of shared_b: {shared_b}")

if __name__ == "__main__":
    main()

Final value of shared_a: 0
Final value of shared_b: 0


### 조건 변수와 모니터
- 조건 변수: 특정 조건이 만족될 때까지 대기하거나, 조건이 만족되었음을 알리는 데 사용되는 동기화 도구
    - wait(): 프로세스가 실행할 조건이 되지 않았을 때, 실행을 중단시키는 도구
    - signal(): 프로세스를 실행할 조건을 만족하여, 대기 중인 프로세스를 깨우는 도구
- 모니터: 조건 변수와 뮤텍스를 결합한 고수준의 동기화 도구
    - 공유자원을 다루는 인터페이스에 접근하기 위한 큐를 통해,
    - 하나의 프로세스만 모니터 내에 진입 가능한 동기화 도구
    - ![](https://velog.velcdn.com/images/bzeromo/post/fbab1ff2-e591-4795-92cd-1e4559eb5414/image.png)
- 뮤텍스, 세마포어, 모니터에 대한 옷 가게 비유
    - 뮤텍스는 탈의실이 하나 있는 옷 가게와 같아서, 한 번에 한 사람만 들어갈 수 있다.
    - 세마포어는 탈의실이 여러 개 있는 옷 가게와 같아서, 여러 사람이 동시에 각각 다른 탈의실에 들어갈 수 있다.
    - 모니터는 유니클로 탈의실과 같아서, 탈의실을 가기 위해 줄(큐)을 서서 기다리고 직원(인터페이스)에게 번호표를 받은 다음, 탈의실에 입장할 수 있다.
- 스레드 안전: 여러 스레드가 동시에 실행되어도 프로그램의 실행에 문제가 없는 상태
    - 스레드 안전한 함수: 여러 스레드가 동시에 호출되어도 문제가 없는 함수
    - 스레드 안전한 클래스: 여러 스레드가 동시에 인스턴스를 생성하거나, 메서드를 호출해도 문제가 없는 클래스

## 교착 상태
- 일어나지 않을 사건을 기다리며, 프로세스의 진행이 멈춰버리는 현상

### 교착 상태의 발생 조건
1. 상호 배제(Mutual Exclusion)
    - 자원은 한 번에 하나의 프로세스만 사용할 수 있다.
2. 점유와 대기(Hold and Wait)
    - 최소한 하나의 자원을 점유하고 있으면서, 다른 자원을 기다리는 프로세스가 존재한다.
3. 비선점(Non-preemption)
    - 자원을 강제로 빼앗을 수 없다. 자원을 점유한 프로세스가 자원을 반납할 때까지 기다려야 한다.
4. 원형 대기(Circular Wait)
    - 자원을 기다리는 프로세스들이 원형으로 연결되어 있다. 즉, 각 프로세스는 다음 프로세스가 점유한 자원을 기다리고 있다.
![](https://csnote.net/assets/img/os/deadlock.png)

### 교착 상태의 해결 방법
- 교착 상태 예방: 교착 상태를 일으키는 4가지 조건 중 하나를 제거
- 교착 상태 회피: 교착 상태가 발생하지 않도록, 자원을 할당하는 방법
    - 한정된 자원을 무분별하게 할당하지 않음
    - 참고: 은행가 알고리즘(Banker's Algorithm) -> 하단 만화 참고
- 교착 상태 검출 후 회복: 교착 상태가 발생했는지 주기적으로 검사하고, 발생했다면 회복 (프로세스 종료, 자원 강제 회수 등)

## 참고자료
- https://csnote.net/
- https://myapollo.com.tw/blog/interview-question-race-condition/
- https://velog.io/@abcdana/%EC%84%B8%EB%A7%88%ED%8F%AC%EC%96%B4%EC%99%80-%EB%AE%A4%ED%85%8D%EC%8A%A4-IPC
- https://m.blog.naver.com/adamdoha/222021830969
- https://velog.io/@bzeromo/CS-%EC%9A%B4%EC%98%81%EC%B2%B4%EC%A0%9C-3
- https://velog.io/@minu-j/%EC%9A%B4%EC%98%81%EC%B2%B4%EC%A0%9C-%EB%A7%8C%ED%99%94%EB%A1%9C-%EC%95%8C%EC%95%84%EB%B3%B4%EB%8A%94-%EC%9D%80%ED%96%89%EC%9B%90-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-%EA%B5%90%EC%B0%A9%EC%83%81%ED%83%9C-%ED%9A%8C%ED%94%BC-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98
    - ![](https://velog.velcdn.com/images/minu-j/post/71f8644c-40ea-488c-9654-b3c1be2518d0/image.jpg)